In [4]:
import numpy as np
import pandas as pd
from preamble import *
import matplotlib.pyplot as plt
import koreanize_matplotlib

In [9]:
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#데이터 불러오기
cancer = load_breast_cancer()
#데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=0)
#전처리
scaler = MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
#모델 처리
svm = SVC()
svm.fit(X_train_scaled, y_train)
X_test_scaled = scaler.transform(X_test)
#모델 확인
print("테스트 점수:{:.2f}".format(svm.score(X_test_scaled, y_test)))

테스트 점수:0.97


In [ ]:
# GCV: 완전탐색 (자원이 넉넉할시에만) 주로 RandomizedSearchCV 사용
# from sklearn.model_selection import GridSearchCV
# param_grid = {'C':[0.001,0.01,0.1,1,10,100],
#              'gamma':[0.001,0.01,0.1,1,10,100]}

# grid = GridSearchCV(SVC(), param_grid=param_grid, cv=5)
# grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)

# grid.fit(X_train_scaled,y_train)
# print(grid.best_score_, grid.score(X_test_scaled,y_test), grid.best_params_)

이때까지는 테스트 세트가 이미 스케일드된 훈련된 데이터: 아예 새로운 데이터시 결과치가 다름

## Pipeline
꼭 거쳐야하는 과정을 강제할 시 사용 (ex. image Resampler, MinMaxScaler..) <br>
1. 전처리 강제
2. 뒤의 모델들에도 전처리 강제
3. 원하는 전처리 값을 넣을 수 있을 시 
fit(),predict(),transfrom(),score()을 지원하지 않는 메서드는 파이프라인에서 사용하기 애매하다 <br>

In [10]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([("scaler",MinMaxScaler()),("svm",SVC())]) # 순서교환을 막기위해 tuple

In [12]:
# GCV: 완전탐색 (자원이 넉넉할시에만) 주로 RandomizedSearchCV 사용
from sklearn.model_selection import GridSearchCV
param_grid = {'svm__C':[0.001,0.01,0.1,1,10,100],
             'svm__gamma':[0.001,0.01,0.1,1,10,100]}

# pipeline 사용시 SVC()->pipe
# 쪼개서 넣지 않고 pipe로 묶어서 돌려서 검증
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)

grid.fit(X_train,y_train)
print(grid.best_score_, grid.score(X_test,y_test), grid.best_params_)

0.9812311901504789 0.972027972027972 {'svm__C': 1, 'svm__gamma': 1}
